In [8]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Extracting features mfcc, chroma, and mel
# FUnction has 4 parameters,
# - File name, 3Boolean for the three features.
# -mfcc - Mel Frequency Cepstral # Coefficient, short term power spectrum of sound
# chroma - THe 12 different pitch classes
# mel - Mel Spectrogram Frequency
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        x = sound_file.read(dtype='float32')
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(x))
            result = np.array([])
        if mfcc:
            mfccs= np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft,sr=sample_rate).T, axis=0)
            result = np.hstack((result,chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(x,sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
        return result
emotions = {
    '01': 'neutral', '02':'calm','03':'happy','04':'sad','05':'angry','06':'fearful','07':'disgust','08':'surprised'
}

observed_emotions = ['calm','happy','fearful','disgust']


# Loading data
def load_data(test_size=0.2):
    x,y = [],[]
    for file in glob.glob("./data/Actor_*/*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x),y,test_size=test_size,random_state=9)

# Splitting the data
x_train, x_test, y_train, y_test = load_data(test_size=0.25)
print(f'Features extracted: {x_train.shape[1]}')

# Initializing the MLPclassifier
model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

# Fitting the model
model.fit(x_train,y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))


Features extracted: 180
Accuracy: 72.40%
